# **1.  Phát hiện Javascript bị rối mã**

In [1]:
# Import necessary libraries
import os
from sklearn.feature_extraction.text import HashingVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline

In [3]:
# Định nghĩa đường dẫn 2 thư mục Javascipts đã giải nén. 
js_path = "./dataset/JavascriptSamples"
obfuscated_js_path = "./dataset/JavascriptSamplesObfuscated"
corpus = []
labels = []
file_types_and_labels = [(js_path, 0), (obfuscated_js_path, 1)]

In [4]:
# Tiếp theo ta sẽ gán nhãn cho chúng.
for files_path, label in file_types_and_labels:
  files = os.listdir(files_path)
  print(files)
  for file in files:
    file_path = files_path + "/" + file
    try:
      with open(file_path, "r") as myfile:
        data = myfile.read().replace("\n", "")
        data = str(data)
        corpus.append(data)
        labels.append(label)
    except:
      pass

['ng-jq-obfuscated.spec.js', 'ng-jq.spec.js', 'ng-route-promise-obfuscated.spec.js', 'ng-route-promise.spec.js', 'ngInit-obfuscated.js', 'ngInit.js', 'ngInitSpec-obfuscated.js', 'ngInitSpec.js', 'ngKeySpec-obfuscated.js', 'ngKeySpec.js', 'ngList-obfuscated.js', 'ngList.js', 'ngListSpec-obfuscated.js', 'ngListSpec.js', 'ngModel-obfuscated.js', 'ngModel.js', 'ngModelOptions-obfuscated.js', 'ngModelOptions.js', 'ngModelOptionsSpec-obfuscated.js', 'ngModelOptionsSpec.js', 'ngModelSpec-obfuscated.js', 'ngModelSpec.js', 'ngNonBindable-obfuscated.js', 'ngNonBindable.js', 'ngNonBindableSpec-obfuscated.js', 'ngNonBindableSpec.js', 'ngOnSpec-obfuscated.js', 'ngOnSpec.js', 'ngOptions-obfuscated.js', 'ngOptions.js', 'ngOptionsSpec-obfuscated.js', 'ngOptionsSpec.js', 'ngPluralize-obfuscated.js', 'ngPluralize.js', 'ngPluralizeSpec-obfuscated.js', 'ngPluralizeSpec.js', 'ngPropSpec-obfuscated.js', 'ngPropSpec.js', 'ngRef-obfuscated.js', 'ngRef.js', 'ngRefSpec-obfuscated.js', 'ngRefSpec.js', 'ngRepeat-

In [6]:
# Ta chia tập dữ liệu thành tập huấn luyện và tập thử nghiệm, đồng thời tạo pipeline cho NLP, tiếp theo sử dụng phân loại random forest.
X_train, X_test, y_train, y_test = train_test_split(
  corpus, 
  labels, 
  test_size=0.33, 
  random_state=42
)
text_clf = Pipeline(
  [
    ("vect", HashingVectorizer(
        input="content", 
        ngram_range=(1, 3)
        )
    ), 
    ("tfidf", TfidfTransformer(use_idf=True,)), # Calcute the importance of a token in a document. For more details: https://viblo.asia/p/tf-idf-term-frequency-inverse-document-frequency-JQVkVZgKkyd
    ("rf", RandomForestClassifier(class_weight="balanced")), # Use 'Random Forest' algorithm for training
  ]
)

In [7]:
# Sau đó chạy huấn luyện và cho ra đánh giá.
from sklearn.feature_extraction.text import HashingVectorizer
corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]
vectorizer = HashingVectorizer(n_features=2**4)
X = vectorizer.fit_transform(corpus)
X.data

array([-0.57735027, -0.57735027,  0.57735027,  0.        , -0.81649658,
        0.40824829,  0.40824829,  0.        , -0.70710678,  0.70710678,
        0.        ,  0.        , -0.57735027, -0.57735027,  0.57735027,
        0.        ])

# **TASK 1: Cho biết kết quả accuracy và confusion matrix**

In [14]:
text_clf.fit(X_train, y_train)

Pipeline(steps=[('vect', HashingVectorizer(ngram_range=(1, 3))),
                ('tfidf', TfidfTransformer()),
                ('rf', RandomForestClassifier(class_weight='balanced'))])

In [15]:
y_test_pred = text_clf.predict(X_test)
print("Accuracy Score: %s" % accuracy_score(y_test, y_test_pred))
print("Confusion matrix: \n %s" % confusion_matrix(y_test, y_test_pred))

Accuracy Score: 0.9705882352941176
Confusion matrix: 
 [[611  16]
 [ 14 379]]


# **2.  Trích xuất thuộc tính tập tin PDF**

In [51]:
# Import IPython để thu thập các output của script. 
from IPython.utils import io

In [52]:
#  Định nghĩa hàm trích xuất thuộc tính. Chạy pdfid đọc một tập và lấy kết quả output của chúng. Kế tiếp, phân tích output để lấy vector số
def PDF_to_FV(file_path):
  """Featurize a PDF file using pdfid."""
  with io.capture_output() as captured:
    %run -i pdfid $file_path 
    # '%run' is a magic function of the loaded module. It execute `pdfid` command syntax with '-i' option to ignore 'sys.exit()'
    # More reference at: https://ipython.org/ipython-doc/dev/interactive/magics.html#magic-run
  print(captured) 
  out = captured.stdout
  out1 = out.split("\n")[2:-2]
  return [int(x.split()[-1]) for x in out1]

In [53]:
# Import listdir để liệt kê các tập tin của thư mục PDF
from os import listdir 
PDFs_path = "/home/wanthinnn/Documents/NT522/NT522_Machine-learning-for-Information-Security/Labs/Lab_3/PDFSamples"


In [54]:
%cd "Tool/pdfid_v0_2_10"
!ls

[Errno 2] No such file or directory: 'Tool/pdfid_v0_2_10'
/home/wanthinnn/Documents/NT522/NT522_Machine-learning-for-Information-Security/Labs/Lab_3/Tool/pdfid_v0_2_10
pdfid.ini
pdfid.ini:Zone.Identifier
pdfid.py
pdfid.py:Zone.Identifier
plugin_embeddedfile.py
plugin_embeddedfile.py:Zone.Identifier
plugin_list
plugin_list:Zone.Identifier
plugin_nameobfuscation.py
plugin_nameobfuscation.py:Zone.Identifier
plugin_triage.py
plugin_triage.py:Zone.Identifier


In [55]:
!pwd

/home/wanthinnn/Documents/NT522/NT522_Machine-learning-for-Information-Security/Labs/Lab_3/Tool/pdfid_v0_2_10


In [56]:
#  Cho vào vòng lặp để trích xuất, quét hết tất cả tập tin vào mảng X
X = []
files = listdir(PDFs_path)
print(files)
for file in files:
  try:
    print(PDFs_path)
    file_path = "\"" + PDFs_path + "/" + file + "\""
    print(file_path)
    X.append(PDF_to_FV(file_path))
  except Exception as e:
    print(e)
print(X)

['PythonBrochure.pdf', 'pdfDOCS_User_Reference_Guide-1.pdf']
/home/wanthinnn/Documents/NT522/NT522_Machine-learning-for-Information-Security/Labs/Lab_3/PDFSamples
"/home/wanthinnn/Documents/NT522/NT522_Machine-learning-for-Information-Security/Labs/Lab_3/PDFSamples/PythonBrochure.pdf"
PDFiD 0.2.10 /home/wanthinnn/Documents/NT522/NT522_Machine-learning-for-Information-Security/Labs/Lab_3/PDFSamples/PythonBrochure.pdf
 PDF Header: %PDF-1.6
 obj                 1096
 endobj              1095
 stream              1061
 endstream           1061
 xref                   0
 trailer                0
 startxref              2
 /Page                 32
 /Encrypt               0
 /ObjStm               43
 /JS                    0
 /JavaScript            0
 /AA                    1
 /OpenAction            0
 /AcroForm              1
 /JBIG2Decode           0
 /RichMedia             0
 /Launch                0
 /EmbeddedFile          0
 /XFA                   0
 /URI                   0
 /Colors > 2

# **Task 2: Cho biết kết quả vector X**

- **File thứ nhất:**
```bash
['PythonBrochure.pdf', 'pdfDOCS_User_Reference_Guide-1.pdf']
/home/wanthinnn/Documents/NT522/NT522_Machine-learning-for-Information-Security/Labs/Lab_3/PDFSamples
"/home/wanthinnn/Documents/NT522/NT522_Machine-learning-for-Information-Security/Labs/Lab_3/PDFSamples/PythonBrochure.pdf"
PDFiD 0.2.10 /home/wanthinnn/Documents/NT522/NT522_Machine-learning-for-Information-Security/Labs/Lab_3/PDFSamples/PythonBrochure.pdf
 PDF Header: %PDF-1.6
 obj                 1096
 endobj              1095
 stream              1061
 endstream           1061
 xref                   0
 trailer                0
 startxref              2
 /Page                 32
 /Encrypt               0
 /ObjStm               43
 /JS                    0
 /JavaScript            0
 /AA                    1
 /OpenAction            0
 /AcroForm              1
 /JBIG2Decode           0
 /RichMedia             0
 /Launch                0
 /EmbeddedFile          0
 /XFA                   0
 /URI                   0
 /Colors > 2^24         0
```


- **File thứ hai:**
```bash
/home/wanthinnn/Documents/NT522/NT522_Machine-learning-for-Information-Security/Labs/Lab_3/PDFSamples
"/home/wanthinnn/Documents/NT522/NT522_Machine-learning-for-Information-Security/Labs/Lab_3/PDFSamples/pdfDOCS_User_Reference_Guide-1.pdf"
PDFiD 0.2.10 /home/wanthinnn/Documents/NT522/NT522_Machine-learning-for-Information-Security/Labs/Lab_3/PDFSamples/pdfDOCS_User_Reference_Guide-1.pdf
 PDF Header: %PDF-1.6
 obj                  153
 endobj               153
 stream                82
 endstream             82
 xref                   2
 trailer                2
 startxref              2
 /Page                  7
 /Encrypt               0
 /ObjStm                0
 /JS                    0
 /JavaScript            0
 /AA                    0
 /OpenAction            0
 /AcroForm              2
 /JBIG2Decode           0
 /RichMedia             0
 /Launch                0
 /EmbeddedFile          0
 /XFA                   0
 /URI                   0
 /Colors > 2^24         0
```

- **2 mảng số lượng thuộc tính trả về:**
```bash
[[1096, 1095, 1061, 1061, 0, 0, 2, 32, 0, 43, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0], [153, 153, 82, 82, 2, 2, 2, 7, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0]]
```

- **Ý nghĩa**: Mỗi phần tử trong mảng X đại diện cho vector đặc trưng (feature vector) của một file PDF, với các giá trị là số lần xuất hiện của các đối tượng như:
    + obj, endobj, stream, endstream, xref, trailer, startxref
    + các keyword như /Page, /Encrypt, /ObjStm, /JS, /JavaScript, ...
    + các tính năng nguy hiểm như /Launch, /EmbeddedFile, /OpenAction, v.v.

# **3. Trích xuất N-grams bằng cách sử dụng thuật toán hash-gram**

In [8]:
#  Chỉ định thư mục cần trích xuất, tham số N, import thư viện để hash và trích xuất N-grams từ chuỗi.
from os import listdir 
from nltk import ngrams 
import hashlib 
 
directories = ["./dataset/Benign PE Samples", "./dataset/Malicious PE Samples"] 
N = 2 

In [9]:
# Tạo các hàm đọc tập tin và chuyển chúng thành N-grams. 
def read_file(file_path): 
    """Reads in the binary sequence of a binary file.""" 
    with open(file_path, "rb") as binary_file: 
        data = binary_file.read() 
    return data 
 
def byte_sequence_to_Ngrams(byte_sequence, N): 
    """Creates a list of N-grams from a byte sequence.""" 
    return ngrams(byte_sequence, N)

In [10]:
# Tiến hành hash N-grams. 
def hash_input(inp): 
 """Compute the MD5 hash of an input.""" 
 return int(hashlib.md5(inp).hexdigest(), 16) 
  
def make_ngram_hashable(Ngram): 
 """Convert N-gram into bytes to be hashable.""" 
 return bytes(Ngram)

In [11]:
'''
Hàm hash_file_Ngrams_into_dictionary lấy một N-grams, hash nó, sau đó tăng số 
lượng count trong dict cho hàm băm. Module B đảm bảo không thể có nhiều hơn 
B khoá trong dict. 
'''
def hash_file_Ngrams_into_dictionary(file_Ngrams, T): 
    """Hashes N-grams in a list and then keeps track of the counts in a 
dictionary.""" 
    for Ngram in file_Ngrams: 
        hashable_Ngram = make_ngram_hashable(Ngram) 
        hashed_and_reduced = hash_input(hashable_Ngram) % B 
        T[hashed_and_reduced] = T.get(hashed_and_reduced, 0) + 1

In [12]:
# Giá trị B là số nguyên tố lớn nhất nhỏ hơn 2^16 và tạo dict rỗng. Tiếp theo lặp lại qua các tập tin để count N-grams đã hash
B = 65521 
T = {}
for dataset_path in directories: 
    samples = [f for f in listdir(dataset_path)] 
    for file in samples: 
        file_path = dataset_path + "/" + file 
        file_byte_sequence = read_file(file_path) 
        file_Ngrams = byte_sequence_to_Ngrams(file_byte_sequence, N) 
        hash_file_Ngrams_into_dictionary(file_Ngrams, T)

In [ ]:
# Sau khi chọn top N-grams được băm, ta tạo bộ thuộc tính N-grams, làm tăng vector đặc trưng. 
def featurize_sample(file, K1_most_common_Ngrams_Using_Hash_Grams): 
    """Takes a sample and produces a feature vector. 
    The features are the counts of the K1 N-grams we've selected. 
    """ 
    K1 = len(K1_most_common_Ngrams_Using_Hash_Grams) 
    fv = K1 * [0] 
    file_byte_sequence = read_file(file_path) 
    file_Ngrams = byte_sequence_to_Ngrams(file_byte_sequence, N) 
    for Ngram in file_Ngrams: 
        hashable_Ngram = make_ngram_hashable(Ngram) 
        hashed_and_reduced = hash_input(hashable_Ngram) % B 
        if hashed_and_reduced in K1_most_common_Ngrams_Using_Hash_Grams: 
            index = K1_most_common_Ngrams_Using_Hash_Grams.index(hashed_and_reduced) 
            fv[index] += 1 
    return fv

In [13]:
# Tạ chọn 1000 N-gram phổ biến sử dụng với heapq. 
K1 = 1000 
import heapq 
K1_most_common_Ngrams_Using_Hash_Grams = heapq.nlargest(K1, T)

In [16]:
# Cuối cùng tạo bộ dữ liệu. 
X = []
for dataset_path in directories: 
  samples = [f for f in listdir(dataset_path)] 
for file in samples: 
  file_path = dataset_path + "/" + file 
  X.append(featurize_sample(file_path, K1_most_common_Ngrams_Using_Hash_Grams))

# **Task 3: Cho biết kết quả vector X**

In [30]:
X


[[11,
  6,
  7,
  23,
  10,
  10,
  13,
  18,
  6,
  12,
  3,
  7,
  6,
  13,
  14,
  8,
  8,
  8,
  6,
  7,
  17,
  18,
  9,
  15,
  9,
  8,
  5,
  23,
  11,
  7,
  8,
  2,
  9,
  16,
  9,
  3,
  5,
  18,
  11,
  8,
  4,
  13,
  3,
  6,
  2,
  5,
  6,
  17,
  12,
  9,
  14,
  4,
  11,
  6,
  14,
  6,
  8,
  4,
  3,
  3,
  6,
  8,
  15,
  13,
  6,
  12,
  15,
  5,
  8,
  8,
  9,
  6,
  11,
  11,
  3,
  17,
  16,
  11,
  13,
  12,
  27,
  4,
  5,
  6,
  8,
  9,
  8,
  9,
  16,
  19,
  13,
  18,
  19,
  1,
  7,
  7,
  4,
  4,
  11,
  15,
  13,
  7,
  9,
  15,
  10,
  10,
  7,
  23,
  6,
  11,
  11,
  7,
  23,
  5,
  21,
  26,
  17,
  3,
  12,
  18,
  4,
  9,
  33,
  7,
  10,
  15,
  5,
  22,
  6,
  5,
  9,
  6,
  10,
  13,
  21,
  16,
  7,
  8,
  5,
  4,
  14,
  10,
  3,
  4,
  4,
  8,
  26,
  12,
  6,
  8,
  8,
  3,
  13,
  19,
  1,
  4,
  5,
  9,
  12,
  4,
  11,
  30,
  9,
  12,
  9,
  3,
  7,
  5,
  7,
  6,
  13,
  3,
  6,
  23,
  6,
  3,
  4,
  4,
  3,
  26,
  4,
  15,
  8,
  18,
  

In [ ]:
print(len(X))
print(len(X[0]))

15
1000


- Dựa trên toàn bộ quá trình trích xuất và xử lý N-grams bằng thuật toán hash-gram, vector X là một tập hợp gồm 15 vector (tương ứng với 15 file mẫu được xử lý), mỗi vector có 1000 chiều (do ta chọn top 1000 N-grams phổ biến nhất trên toàn bộ dữ liệu).
- Cụ thể, mỗi vector trong X (ví dụ như X[0] cho file mẫu thứ nhất) chứa 1000 số nguyên, trong đó mỗi giá trị biểu thị số lần xuất hiện của một N-gram (sau khi được băm qua thuật toán MD5 và giảm modulo B với B = 65521). 
- Kết quả in ra cho thấy:
    + Tổng số file được xử lý: 15 (print(len(X)) = 15)
    + Kích thước của vector đặc trưng cho mỗi file: 1000 chiều (print(len(X[0])) = 1000)
- Như vậy, vector X có dạng như bên dưới. Trong đó, mỗi fv_i_j là số đếm của N-gram thứ j (trong danh sách top 1000 đã chọn) xuất hiện trong file mẫu thứ i.
```python
X = [
    [fv_1_1, fv_1_2, ..., fv_1_1000],   # File mẫu 1
    [fv_2_1, fv_2_2, ..., fv_2_1000],   # File mẫu 2
    ...
    [fv_15_1, fv_15_2, ..., fv_15_1000]   # File mẫu 15
]
```
- Kết hợp với các thông tin trên, đáp án cho câu 3 "Cho biết kết quả vector X" như sau: Vector X là tập hợp của 15 vector, mỗi vector có 1000 phần tử, trong đó các phần tử thể hiện số lượng xuất hiện của các N-grams (đã được hash và giảm modulo 65521) trích xuất từ các file mẫu trong thư mục dataset.


# **4.  Xây dựng bộ phân loại động phần mềm độc hại**

In [1]:
# Do tập dữ liệu là các tập tin nhật ký định JSON nên import thư viện json
import numpy as np 
import os 
import json 
 
directories_with_labels = [("./dataset/DA Logs Benign", 0), ("./dataset/DA Logs Malware", 
1)]

In [3]:
# Viết hàm parse nhật ký JSON. 
def get_API_class_method_type_from_log(log): 
    """Parses out API calls from behavioral logs.""" 
    API_data_sequence = [] 
    with open(log) as log_file: 
        json_log = json.load(log_file) 
        api_calls_array = "[" + json_log["api_calls"] + "]"
        # Chọn trích xuất class, method và type từ API call. 
        api_calls = json.loads(api_calls_array) 
        for api_call in api_calls: 
            data = api_call["class"] + ":" + api_call["method"] + ":" + api_call["type"] 
            API_data_sequence.append(data) 
    return API_data_sequence

In [4]:
# Sau đó phân nhãn. 
data_corpus = [] 
labels = [] 
for directory, label in directories_with_labels: 
    logs = os.listdir(directory) 
    for log_path in logs: 
        file_path = directory + "/" + log_path 
        try: 
            data_corpus.append(get_API_class_method_type_from_log(file_path)) 
            labels.append(label) 
        except: 
            pass 
print(data_corpus[0])

['android.os.SystemProperties:get:content', 'android.os.SystemProperties:get:content', 'android.os.SystemProperties:get:content', 'android.os.SystemProperties:get:content', 'android.os.SystemProperties:get:content', 'android.os.SystemProperties:get:content', 'android.os.SystemProperties:get:content', 'android.os.SystemProperties:get:content', 'android.os.SystemProperties:get:content', 'android.os.SystemProperties:get:content', 'android.os.SystemProperties:get:content', 'android.os.SystemProperties:get:content', 'android.app.ContextImpl:registerReceiver:binder', 'android.app.ContextImpl:registerReceiver:binder', 'android.os.SystemProperties:get:content', 'android.os.SystemProperties:get:content']


In [5]:
# Chia tập dữ liệu huấn luyện và kiểm thử 
from sklearn.model_selection import train_test_split 
 
corpus_train, corpus_test, y_train, y_test = train_test_split( 
    data_corpus, labels, test_size=0.2, random_state=11 
)

In [8]:
# Trích xuất N-grams.
import collections 
from nltk import ngrams 
import numpy as np 
 
def read_file(file_path): 
    """Reads in the binary sequence of a binary file.""" 
    with open(file_path, "rb") as binary_file: 
        data = binary_file.read() 
    return data 
 
def text_to_Ngrams(text, n): 
    """Produces a list of N-grams from a text.""" 
    Ngrams = ngrams(text, n) 
    return list(Ngrams)

def get_Ngram_counts(text, N): 
    """Get a frequency count of N-grams in a text.""" 
    Ngrams = text_to_Ngrams(text, N) 
    return collections.Counter(Ngrams)

In [9]:
# Xác định N=4 và thu thập tất cả N-grams. 
N = 4 
total_Ngram_count = collections.Counter([]) 
for file in corpus_train: 
    total_Ngram_count += get_Ngram_counts(file, N)

In [10]:
# Sau đó thu hẹp K1=3000 các N-grams phổ biến. 
K1 = 3000 
K1_most_frequent_Ngrams = total_Ngram_count.most_common(K1) 
K1_most_frequent_Ngrams_list = [x[0] for x in K1_most_frequent_Ngrams]

In [11]:
# Viết một hàm phân lại một mẫu thành một vector có N-grams. 
def featurize_sample(file, Ngrams_list): 
    """Takes a sample and produces a feature vector. 
    The features are the counts of the K1 N-grams we've selected. 
    """ 
    K1 = len(Ngrams_list) 
    feature_vector = K1 * [0] 
    fileNgrams = get_Ngram_counts(file, N) 
    for i in range(K1): 
        feature_vector[i] = fileNgrams[Ngrams_list[i]] 
    return feature_vector

In [12]:
#  Sử dụng hàm trên để tạo bộ huấn luyện và mẫu kiểm thử. 
X_train = [] 
for sample in corpus_train: 
    X_train.append(featurize_sample(sample, 
K1_most_frequent_Ngrams_list)) 
X_train = np.asarray(X_train) 
X_test = [] 
for sample in corpus_test:
    X_test.append(featurize_sample(sample, 
K1_most_frequent_Ngrams_list)) 
X_test = np.asarray(X_test) 

In [13]:
# Tiếp tục thu hẹp K1=3000 N-12 grams thành K2=500. Sau đó thiết lập pipeline 
# chạy phân loại XGBoost. 
from sklearn.feature_selection import SelectKBest, mutual_info_classif 
from sklearn.pipeline import Pipeline 
from xgboost import XGBClassifier 
 
K2 = 500 
mi_pipeline = Pipeline( 
    [ 
        ("mutual_information", SelectKBest(mutual_info_classif, k=K2)), 
        ("xgb", XGBClassifier()), 
    ] 
)

In [14]:
# Sau đó huấn luyện và kiểm thử tập train, test. 
mi_pipeline.fit(X_train, y_train) 
print("Training accuracy:") 
print(mi_pipeline.score(X_train, y_train)) 
print("Testing accuracy:") 
print(mi_pipeline.score(X_test, y_test))

Training accuracy:
0.8998797354179194
Testing accuracy:
0.8436560432952496


# **Task 4: Cho biết kết quả đánh giá.**
Kết quả đánh giá mô hình như sau:
- Độ chính xác trên tập huấn luyện (Training accuracy): 0.8999 ≈ 89.99%
- Độ chính xác trên tập kiểm thử (Testing accuracy): 0.8437 ≈ 84.37%

=> Nhận xét: mô hình đạt độ chính xác cao trên tập huấn luyện, và vẫn giữ được hiệu năng tốt (~84.4%) trên dữ liệu chưa thấy trước, cho thấy khả năng tổng quát hoá khá ổn — tuy có hơi chênh lệch giữa train và test, gợi ý một chút over‑fitting nhẹ.

# **5. MalConv – Quy trình áp dụng sâu cho phát hiện phần mềm độc hại PE**

In [27]:
# Import thư viên numpy để tính toán vector và tqdm để theo dõi tiền trình trong vòng lặp. 
import numpy as np 
from tqdm import tqdm

In [28]:
# Định nghĩa hàm để chuyển byte thành vector. 
def embed_bytes(byte): 
    binary_string = "{0:08b}".format(byte) 
    vec = np.zeros(8) 
    for i in range(8): 
        if binary_string[i] == "1": 
            vec[i] = float(1) / 16 
        else: 
            vec[i] = -float(1) / 16 
    return vec

In [29]:
# Đọc các tin PE mẫu và dán nhãn cho chúng. 
import os 
from os import listdir 
 
directories_with_labels  =  [("./dataset/Benign PE Samples",  0),  ("./dataset/Malicious PE Samples", 1)] 
list_of_samples = [] 
labels = [] 
for dataset_path, label in directories_with_labels: 
    samples = [f for f in listdir(dataset_path)] 
    for file in samples: 
        file_path = os.path.join(dataset_path, file) 
        list_of_samples.append(file_path)
        labels.append(label)

In [30]:
# Định nghĩa hàm đọc chuỗi byte trong tập tin. 
def read_file(file_path): 
    """Read the binary sequence of a file.""" 
    with open(file_path, "rb") as binary_file: 
        return binary_file.read()

In [31]:
# Đặt độ dài tối đa, maxSize byte, để đọc cho mỗi mẫu, lấy tất cả byte của mẫu đưa vào X. 
max_size = 15000 
num_samples = len(list_of_samples) 
X = np.zeros((num_samples, 8, max_size)) 
Y = np.asarray(labels) 
file_num = 0 
for file in tqdm(list_of_samples): 
    sample_byte_sequence = read_file(file) 
    for i in range(min(max_size, len(sample_byte_sequence))): 
        X[file_num, :, i] = embed_bytes(sample_byte_sequence[i]) 
    file_num += 1

100%|██████████| 393/393 [00:12<00:00, 32.21it/s]


In [32]:
# Thiết lập trình tối ưu. 
from keras import optimizers
my_opt = optimizers.SGD(learning_rate=0.01, decay=1e-5, nesterov=True)

In [33]:
# Sử dụng API của Keras để thiết lập màn thần kinh học sâu. 
from keras import Input 
inputs = Input(shape=(8, max_size)) 
 
from keras.layers import Conv1D 
conv1 = Conv1D(kernel_size=(128), filters=32, strides=(128), 
padding="same")(inputs) 
conv2 = Conv1D(kernel_size=(128), filters=32, strides=(128), 
padding="same")(inputs) 
 
from keras.layers import Activation 
a = Activation("sigmoid", name="sigmoid")(conv2)
from keras.layers import multiply 
mul = multiply([conv1, a]) 
b = Activation("relu", name="relu")(mul) 
 
from keras.layers import GlobalMaxPool1D 
p = GlobalMaxPool1D()(b) 
 
from keras.layers import Dense 
d = Dense(16)(p) 
predictions = Dense(1, activation="sigmoid")(d) 
 
from keras import Model 
model = Model(inputs=inputs, outputs=predictions) 

In [34]:
# Biên dịch mô hình và chọn batch size. 
model.compile(optimizer=my_opt, loss="binary_crossentropy", 
metrics=["acc"]) 
model.summary() 
 
batch_size = 16 
num_batches = int(num_samples / batch_size)



Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 8, 15000)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 1, 32)     │ 61,440,032 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 1, 32)     │ 61,440,032 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sigmoid             │ (None, 1, 32)     │          0 │ conv1d_3[0][0]    │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_1          │ (None, 1, 32)     │          0 │ conv1d_2[0][0],   │
│ (Multiply)          │                   │            │ sigmoid[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ relu (Activation)   │ (None, 1, 32)     │          0 │ multiply_1[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 32)        │          0 │ relu[0][0]        │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 16)        │        528 │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 1)         │         17 │ dense_2[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 122,880,609 (468.75 MB)

 Trainable params: 122,880,609 (468.75 MB)

 Non-trainable params: 0 (0.00 B)

In [35]:
# Huấn luyện mô hình. 
for batch_num in tqdm(range(num_batches)): 
    batch = X[batch_num * batch_size : (batch_num + 1) * batch_size] 
    model.train_on_batch( 
        batch, Y[batch_num * batch_size : (batch_num + 1) * batch_size] 
    )

100%|██████████| 24/24 [00:16<00:00,  1.42it/s]


In [36]:
# Đánh giá mô hình.
print(model.evaluate(X, Y))

13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 77ms/step - acc: 0.9934 - loss: 0.2884
[0.3249645531177521, 0.9618320465087891]


# **Task 5. Cho biết kết quả đánh giá mô hình qua tập test**
- Mô hình MalConv đạt độ chính xác ~96.18%, cho thấy khả năng phân biệt phần mềm độc hại và hợp lệ khá tốt.

- Mặc dù loss (≈0.32) vẫn còn tương đối, nhưng độ chính xác cao chứng tỏ mô hình học được đặc trưng tốt.

# **6. Xử lý phần mềm độc hại packer**

In [37]:
import os 
 
files_path = "./dataset/Benign PE Samples UPX/" 
files = os.listdir(files_path) 
file_paths = [files_path+x for x in files]

In [38]:
# Chạy UPX để nén các tập tin PE.
from subprocess import Popen, PIPE 
 
cmd = "./tool/upx-5.0.0-win64/upx.exe" 
for path in file_paths: 
    cmd2 = cmd+" \""+path+"\"" 
    res = Popen(cmd2, stdout=PIPE).communicate() 
    print(res) 
    if "error" in str(res[0]): 
        print(path) 
        os.remove(path) 

(b'                       Ultimate Packer for eXecutables\r\n                          Copyright (C) 1996 - 2025\r\nUPX 5.0.0       Markus Oberhumer, Laszlo Molnar & John Reiser   Feb 20th 2025\r\n\r\n        File size         Ratio      Format      Name\r\n   --------------------   ------   -----------   -----------\r\n\r\nPacked 1 file: 0 ok, 1 error.\r\n', None)
./dataset/Benign PE Samples UPX/alg.exe
(b'                       Ultimate Packer for eXecutables\r\n                          Copyright (C) 1996 - 2025\r\nUPX 5.0.0       Markus Oberhumer, Laszlo Molnar & John Reiser   Feb 20th 2025\r\n\r\n        File size         Ratio      Format      Name\r\n   --------------------   ------   -----------   -----------\r\n\r\nPacked 1 file: 0 ok, 1 error.\r\n', None)
./dataset/Benign PE Samples UPX/appidcertstorecheck.exe
(b'                       Ultimate Packer for eXecutables\r\n                          Copyright (C) 1996 - 2025\r\nUPX 5.0.0       Markus Oberhumer, Laszlo Molnar & Jo

# **7. Xây dựng  bộ phân loại packer**

In [40]:
# Đọc tất cả tập tin cần phân tích và gán nhãn cho chúng. 
import os 
from os import listdir 
 
directories_with_labels = [ 
    ("./dataset/Benign PE Samples", 0), 
    ("./dataset/Benign PE Samples UPX", 1), 
    ("./dataset/Benign PE Samples Amber", 2), 
] 
list_of_samples = [] 
labels = [] 
for dataset_path, label in directories_with_labels: 
    samples = [f for f in listdir(dataset_path)] 
    for file in samples: 
        file_path = os.path.join(dataset_path, file) 
        list_of_samples.append(file_path) 
        labels.append(label)

In [42]:
# Phân ra train test. 
from sklearn.model_selection import train_test_split 
 
samples_train, samples_test, labels_train, labels_test = train_test_split ( 
    list_of_samples, labels, test_size=0.3, stratify=labels, random_state=11 
)

In [43]:
# Import thư viện cần thiết để trích xuất N-grams. 
import collections 
from nltk import ngrams 
import numpy as np 

In [44]:
# Định nghĩa hàm sử dụng trích xuất N-grams. 
def read_file(file_path):
    """Reads in the binary sequence of a binary file.""" 
    with open(file_path, "rb") as binary_file: 
        data = binary_file.read() 
    return data 
 
 
def byte_sequence_to_Ngrams(byte_sequence, N): 
    """Creates a list of N-grams from a byte sequence.""" 
    Ngrams = ngrams(byte_sequence, N) 
    return list(Ngrams) 
 
 
def extract_Ngram_counts(file, N): 
    """Takes a binary file and outputs the N-grams counts of its binary 
sequence.""" 
    filebyte_sequence = read_file(file) 
    file_Ngrams = byte_sequence_to_Ngrams(filebyte_sequence, N) 
    return collections.Counter(file_Ngrams) 
 
 
def featurize_sample(sample, K1_most_frequent_Ngrams_list): 
    """Takes a sample and produces a feature vector. 
    The features are the counts of the K1 N-grams we've selected. 
    """ 
    K1 = len(K1_most_frequent_Ngrams_list) 
    feature_vector = K1 * [0] 
    file_Ngrams = extract_Ngram_counts(sample, N) 
    for i in range(K1): 
        feature_vector[i] = file_Ngrams[K1_most_frequent_Ngrams_list[i]] 
    return feature_vector

In [46]:
# Chọn N-grams mong muốn. 
N = 2 
total_Ngram_count = collections.Counter([]) 
for file in samples_train: 
    total_Ngram_count += extract_Ngram_counts(file, N) 
K1 = 100
K1_most_common_Ngrams = total_Ngram_count.most_common(K1) 
K1_most_common_Ngrams_list = [x[0] for x in K1_most_common_Ngrams]

In [48]:
# Thiết lập thuộc tính cho tập test. 
Ngram_features_list_test = [] 
y_test = [] 
for i in range(len(samples_test)): 
    file = samples_test[i] 
    NGram_features = featurize_sample(file, K1_most_common_Ngrams_list) 
    Ngram_features_list_test.append(NGram_features) 
    y_test.append(labels_test[i]) 
X_test = Ngram_features_list_test

In [51]:
# Tạo feature vector cho tập train
Ngram_features_list_train = []
for i in range(len(samples_train)):
    file = samples_train[i]
    NGram_features = featurize_sample(file, K1_most_common_Ngrams_list)
    Ngram_features_list_train.append(NGram_features)

X_train = Ngram_features_list_train
y_train = labels_train


In [52]:
# Huấn luyện mô hình random forest trên tập train. 
from sklearn.ensemble import RandomForestClassifier 
 
clf = RandomForestClassifier(n_estimators=100) 
clf = clf.fit(X_train, y_train) 

In [53]:
# Sử dụng bộ phân loại được đào tạo để dự đoán trên bộ test và đánh đánh giá hiệu suất bằng confusion matrix. 
y_pred = clf.predict(X_test) 
 
from sklearn.metrics import confusion_matrix 
confusion_matrix(y_test, y_pred)

array([[113,   0],
       [  0,  23]], dtype=int64)

In [54]:
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, f1_score, precision_score, roc_auc_score

print("Confusion matrix:\n %s" % confusion_matrix(y_test, y_pred))

# Additional metrics (Not required)
print("Accurcy: %s" %(accuracy_score(y_test, y_pred)))
print("Recall: %s" %(recall_score(y_test, y_pred, average='micro')))
print("Precision: %s" %(precision_score(y_test, y_pred, average='micro')))
print("F1 Score: %s" %(f1_score(y_test, y_pred, average='micro')))
print("ROC AUC: %s" %(roc_auc_score(y_test, y_pred)))

Confusion matrix:
 [[113   0]
 [  0  23]]
Accurcy: 1.0
Recall: 1.0
Precision: 1.0
F1 Score: 1.0
ROC AUC: 1.0


# **Task 7. Cho biết kết quả đánh giá**

Điều này có nghĩa là:
- Confusion matrix: Trong tập kiểm tra, tất cả 113 mẫu của lớp 0 và 23 mẫu của lớp 1 đều được phân loại chính xác, không có mẫu nào bị nhầm lẫn giữa các lớp.
- Accuracy, Recall, Precision, F1 Score, ROC AUC: Tất cả các chỉ số này đều đạt giá trị 1.0, cho thấy mô hình đã dự đoán hoàn hảo không có lỗi nào.

=> Kết quả này chứng tỏ rằng bộ phân loại packer đã thực hiện rất tốt trên dữ liệu kiểm tra. Tuy nhiên, cần lưu ý rằng hiệu suất hoàn hảo thường có thể do tập dữ liệu quá nhỏ hoặc một sự đơn giản trong bài toán phân loại.


# **8. MalGAN – Tạo phần mềm độc hại**

In [18]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [19]:
import os
import pandas as pd
from keras.models import load_model
import MalGAN_utils
import MalGAN_gen_adv_examples

In [21]:
# Xác định các đường dẫn dữ liệu
save_path = "/mnt/c/Users/WanThinnn/Documents/UIT/Nam_3/HK2/NT522_AI-ATTT/NT522_Machine-learning-for-Information-Security/Labs/Lab_3/MalGAN_output" 
model_path = "/mnt/c/Users/WanThinnn/Documents/UIT/Nam_3/HK2/NT522_AI-ATTT/NT522_Machine-learning-for-Information-Security/Labs/Lab_3/MalGAN_input/malconv.h5" 
log_path = "/mnt/c/Users/WanThinnn/Documents/UIT/Nam_3/HK2/NT522_AI-ATTT/NT522_Machine-learning-for-Information-Security/Labs/Lab_3/MalGAN_output/adversarial_log.csv" 
pad_percent = 0.1 
threshold = 0.6 
step_size = 0.01 
limit = 0. 
input_samples = "/mnt/c/Users/WanThinnn/Documents/UIT/Nam_3/HK2/NT522_AI-ATTT/NT522_Machine-learning-for-Information-Security/Labs/Lab_3/MalGAN_input/samplesIn.csv" 


In [ ]:
MalGAN_utils.limit_gpu_memory(limit)

In [23]:
# Đọc tập tin csv chứa name và label của mẫu vào dataframe. 
df = pd.read_csv(input_samples, header=None)
fn_list = df[0].values

for i in range(len(fn_list)):
  fn_list[i] = f'{fn_list[i]}'

for e, fn in enumerate(fn_list):
  print(f'{e}: {fn}')

0: ./dataset/Benign PE Samples/adamuninstall.exe
1: ./dataset/Benign PE Samples/AddInProcess.exe
2: ./dataset/Benign PE Samples/AddInProcess32.exe
3: ./dataset/Benign PE Samples/AddInUtil.exe
4: ./dataset/Benign PE Samples/AddSuggestedFoldersToLibraryDialog.exe
5: ./dataset/Benign PE Samples/ADSchemaAnalyzer.exe
6: ./dataset/Benign PE Samples/AgentService.exe
7: ./dataset/Benign PE Samples/aitstatic.exe
8: ./dataset/Benign PE Samples/alg.exe
9: ./dataset/Benign PE Samples/appcmd.exe
10: ./dataset/Benign PE Samples/AppHostRegistrationVerifier.exe
11: ./dataset/Benign PE Samples/appidcertstorecheck.exe
12: ./dataset/Benign PE Samples/appidpolicyconverter.exe
13: ./dataset/Benign PE Samples/appidtel.exe
14: ./dataset/Benign PE Samples/AppInstaller.exe
15: ./dataset/Benign PE Samples/AppLaunch.exe
16: ./dataset/Benign PE Samples/ApplicationFrameHost.exe
17: ./dataset/Benign PE Samples/ApplySettingsTemplateCatalog.exe
18: ./dataset/Benign PE Samples/ApplyTrustOffline.exe
19: ./dataset/Benig

In [24]:
# Tải mô hình MalConv đã được huấn luyện. 
model = load_model(model_path)
model.summary() # Have an overview of model
# Use 'layers[0]' as input and 'layer[1]' as output for the next snippet code

2025-04-21 01:43:45.660653: W tensorflow/c/c_api.cc:304] Operation '{name:'dense_2_2/kernel/Assign' id:6396 op device:{requested: '', assigned: ''} def:{{{node dense_2_2/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_2_2/kernel, dense_2_2/kernel/Initializer/random_uniform)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2025-04-21 01:43:46.145493: W tensorflow/c/c_api.cc:304] Operation '{name:'conv1d_2_2/kernel/v/Assign' id:6592 op device:{requested: '', assigned: ''} def:{{{node conv1d_2_2/kernel/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](conv1d_2_2/kernel/v, conv1d_2_2/kernel/v/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no 

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 80000)]              0         []                            
                                                                                                  
 embedding_1 (Embedding)     (None, 80000, 8)             2048      ['input_1[0][0]']             
                                                                                                  
 conv1d_2 (Conv1D)           (None, 160, 128)             512128    ['embedding_1[0][0]']         
                                                                                                  
 conv1d_1 (Conv1D)           (None, 160, 128)             512128    ['embedding_1[0][0]']         
                                                                                            

In [25]:
# Sử dụng  Fast Gradient Step Method (FGSM) để tạo mẫu đối kháng. 

adv_samples, log = MalGAN_gen_adv_examples.gen_adv_samples(model, fn_list, pad_percent, step_size, threshold)


2025-04-21 01:43:49.044127: W tensorflow/c/c_api.cc:304] Operation '{name:'embedding_1_2/embedding_lookup/Identity' id:6312 op device:{requested: '', assigned: ''} def:{{{node embedding_1_2/embedding_lookup/Identity}} = Identity[T=DT_FLOAT, _has_manual_control_dependencies=true](embedding_1_2/embedding_lookup)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
/home/wanthinnn/miniconda3/envs/env_wanthinnn/lib/python3.8/site-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2025-04-21 01:43:49.198731: W tensorflow/c/c_api.cc:304] Operation '{name:'dense_2_2/Sigmoid' id:6407 op device:{requested: '', assigned: ''} def:{{{node dense_2_2/Sigmo


FILE: ./dataset/Benign PE Samples/adamuninstall.exe
	file length: 121274 , Exceed max length ! Ignored !
	original score: 0.5159791

FILE: ./dataset/Benign PE Samples/AddInProcess.exe
	file length: 42080
	pad length: 4208
	loss: 0.2581485
	score: 0.25814855
	original score: 0.25814855

FILE: ./dataset/Benign PE Samples/AddInProcess32.exe
	file length: 42080
	pad length: 4208
	loss: nan
	score: nan
	original score: 0.98788273

FILE: ./dataset/Benign PE Samples/AddInUtil.exe
	file length: 42600
	pad length: 4260
	loss: nan
	score: nan
	original score: 0.99946874

FILE: ./dataset/Benign PE Samples/AddSuggestedFoldersToLibraryDialog.exe
	file length: 791456 , Exceed max length ! Ignored !
	original score: 8.399191e-08

FILE: ./dataset/Benign PE Samples/ADSchemaAnalyzer.exe
	file length: 69056
	pad length: 6905
	loss: nan
	score: nan
	original score: 0.9809284

FILE: ./dataset/Benign PE Samples/AgentService.exe
	file length: 1189376 , Exceed max length ! Ignored !
	original score: 3.166838

In [26]:
# Lưu lại log và ghi mẫu mới vào thư mục. 
log.save(log_path) 
for fn, adv in zip(fn_list, adv_samples): 
    _fn = fn.split('/')[-1] 
    dst = os.path.join(save_path, _fn) 
    print(dst) 
    with open(dst, 'wb') as f: 
        f.write(adv)


Log saved to "/mnt/c/Users/WanThinnn/Documents/UIT/Nam_3/HK2/NT522_AI-ATTT/NT522_Machine-learning-for-Information-Security/Labs/Lab_3/MalGAN_output/adversarial_log.csv"

/mnt/c/Users/WanThinnn/Documents/UIT/Nam_3/HK2/NT522_AI-ATTT/NT522_Machine-learning-for-Information-Security/Labs/Lab_3/MalGAN_output/adamuninstall.exe
/mnt/c/Users/WanThinnn/Documents/UIT/Nam_3/HK2/NT522_AI-ATTT/NT522_Machine-learning-for-Information-Security/Labs/Lab_3/MalGAN_output/AddInProcess.exe
/mnt/c/Users/WanThinnn/Documents/UIT/Nam_3/HK2/NT522_AI-ATTT/NT522_Machine-learning-for-Information-Security/Labs/Lab_3/MalGAN_output/AddInProcess32.exe
/mnt/c/Users/WanThinnn/Documents/UIT/Nam_3/HK2/NT522_AI-ATTT/NT522_Machine-learning-for-Information-Security/Labs/Lab_3/MalGAN_output/AddInUtil.exe
/mnt/c/Users/WanThinnn/Documents/UIT/Nam_3/HK2/NT522_AI-ATTT/NT522_Machine-learning-for-Information-Security/Labs/Lab_3/MalGAN_output/AddSuggestedFoldersToLibraryDialog.exe
/mnt/c/Users/WanThinnn/Documents/UIT/Nam_3/HK2/NT522

# **Task 8. Cho biết kết quả đánh giá mẫu mới trong việc đánh lừa bộ nhận diện**
## **Kết quả đánh lừa bộ nhận diện với mẫu mới khá thấp, vì dựa trên các yếu tố sau**

- Đánh giá điểm số:
    + Hầu hết các mẫu đều có điểm ban đầu và điểm dự đoán khá ổn định, tức là mô hình nhận diện đang thực hiện phân loại chính xác đối với hầu hết các tệp.
    + Một số tệp như FileExplorer.exe, fodhelper.exe, fsavailux.exe có điểm số rất thấp và không thay đổi nhiều giữa điểm ban đầu và điểm dự đoán, điều này cho thấy rằng các tệp này được phân loại là hợp pháp hoặc ít có nguy cơ bị đánh lừa.

- Mức độ sai lệch thấp:
    + Những tệp có mức độ sai lệch giữa original score và predict score thấp (như FirstLogonAnim.exe, MiniConfigBuilder.exe) chỉ ra rằng mô hình đánh giá khá chính xác, không bị đánh lừa dễ dàng.
    + Sự thay đổi lớn trong một vài trường hợp như UpdateCheck.exe (điểm gốc thấp nhưng điểm dự đoán rất cao) có thể là kết quả của sự sai lệch trong môi trường thực thi hoặc sự thay đổi bất ngờ trong mẫu thử, nhưng không phải là tình huống phổ biến.

- Tỉ lệ đánh lừa thấp: Mặc dù có một vài mẫu có điểm không chính xác hoặc có sự thay đổi đột ngột trong điểm dự đoán, nhưng đa số mẫu vẫn được phân loại đúng, với điểm ban đầu và điểm dự đoán khá tương đồng. Điều này chỉ ra rằng mô hình hoạt động ổn định và ít bị đánh lừa.

=> Kết luận: Với các phân tích trên, ta có thể kết luận rằng Kết quả đánh lừa bộ nhận diện với mẫu mới khá thấp. Hệ thống có thể phân biệt các tệp độc hại và hợp pháp với độ chính xác cao, mặc dù có một số trường hợp ngoại lệ có thể do yếu tố môi trường hoặc sự thay đổi bất ngờ trong mẫu.